In [1]:
import gym
import gym_Physics_RL
from torch import nn as nn

from rlkit.exploration_strategies.base import PolicyWrappedWithExplorationStrategy
from rlkit.exploration_strategies.epsilon_greedy import EpsilonGreedy
from rlkit.policies.argmax import ArgmaxDiscretePolicy
from rlkit.torch.dqn.double_dqn import DoubleDQNTrainer
from rlkit.torch.networks import Mlp
import rlkit.torch.pytorch_util as ptu
from rlkit.data_management.env_replay_buffer import EnvReplayBuffer
from rlkit.launchers.launcher_util import setup_logger
from rlkit.samplers.data_collector import MdpPathCollector
from rlkit.torch.torch_rl_algorithm import TorchBatchRLAlgorithm


def experiment(variant):
    expl_env = gym.make('Physics_RL-v0')
    eval_env = gym.make('Physics_RL-v0')
    obs_dim = expl_env.observation_space.low.size
    action_dim = eval_env.action_space.n

    qf = Mlp(
        hidden_sizes=[32, 32],
        input_size=obs_dim,
        output_size=action_dim,
    )
    target_qf = Mlp(
        hidden_sizes=[32, 32],
        input_size=obs_dim,
        output_size=action_dim,
    )
    qf_criterion = nn.MSELoss()
    eval_policy = ArgmaxDiscretePolicy(qf)
    expl_policy = PolicyWrappedWithExplorationStrategy(
        EpsilonGreedy(expl_env.action_space),
        eval_policy,
    )
    eval_path_collector = MdpPathCollector(
        eval_env,
        eval_policy,
    )
    expl_path_collector = MdpPathCollector(
        expl_env,
        expl_policy,
    )
    trainer = DoubleDQNTrainer(
        qf=qf,
        target_qf=target_qf,
        qf_criterion=qf_criterion,
        **variant['trainer_kwargs']
    )
    replay_buffer = EnvReplayBuffer(
        variant['replay_buffer_size'],
        expl_env,
    )
    algorithm = TorchBatchRLAlgorithm(
        trainer=trainer,
        exploration_env=expl_env,
        evaluation_env=eval_env,
        exploration_data_collector=expl_path_collector,
        evaluation_data_collector=eval_path_collector,
        replay_buffer=replay_buffer,
        **variant['algorithm_kwargs']
    )
    algorithm.to(ptu.device)
    algorithm.train()

doodad not detected


In [2]:
if __name__ == "__main__":
    # noinspection PyTypeChecker
    variant = dict(
        algorithm="DDQN",
        version="normal",
        layer_size=256,
        replay_buffer_size=int(1E6),
        algorithm_kwargs=dict(
            num_epochs=3000,
            num_eval_steps_per_epoch=5000,
            num_trains_per_train_loop=1000,
            num_expl_steps_per_train_loop=10,
            min_num_steps_before_training=10,
            max_path_length=10,
            batch_size=256,
        ),
        trainer_kwargs=dict(
            discount=0.99,
            learning_rate=3E-4,
        ),
    )
    setup_logger('ddqn-train', variant=variant)
    # ptu.set_gpu_mode(True)  # optionally set the GPU (default=False)
    experiment(variant)

2019-06-08 21:13:54.963605 Pacific Daylight Time | Variant:
2019-06-08 21:13:54.965607 Pacific Daylight Time | {
  "algorithm": "DDQN",
  "version": "normal",
  "layer_size": 256,
  "replay_buffer_size": 1000000,
  "algorithm_kwargs": {
    "num_epochs": 3000,
    "num_eval_steps_per_epoch": 5000,
    "num_trains_per_train_loop": 1000,
    "num_expl_steps_per_train_loop": 10,
    "min_num_steps_before_training": 10,
    "max_path_length": 10,
    "batch_size": 256
  },
  "trainer_kwargs": {
    "discount": 0.99,
    "learning_rate": 0.0003
  }
}
2019-06-08 21:14:03.271335 Pacific Daylight Time | [h:\work\psu\ece 510(deep learning theory and practices)\project\rlkit\rlkit\..\data\ddqn-train\ddqn-train_2019_06_08_21_13_54_0000--s-0] Epoch 0 finished
---------------------------------------------  --------------
replay_buffer/size                               20
trainer/QF Loss                                   3.4821
trainer/Y Predictions Mean                        0.23983
trainer/Y Pre

evaluation/env_infos/X_pos_2 Std                131.014
evaluation/env_infos/X_pos_2 Max                579
evaluation/env_infos/X_pos_2 Min                120
time/data storing (s)                             0.000133017
time/evaluation sampling (s)                      3.95574
time/exploration sampling (s)                     0.00700434
time/logging (s)                                  0.0666458
time/saving (s)                                   0.00952634
time/training (s)                                 4.09406
time/epoch (s)                                    8.13311
time/total (s)                                   10.5608
Epoch                                             0
---------------------------------------------  --------------
2019-06-08 21:14:12.746959 Pacific Daylight Time | [h:\work\psu\ece 510(deep learning theory and practices)\project\rlkit\rlkit\..\data\ddqn-train\ddqn-train_2019_06_08_21_13_54_0000--s-0] Epoch 1 finished
---------------------------------------------

evaluation/env_infos/initial/X_pos_2 Max        580
evaluation/env_infos/initial/X_pos_2 Min        121
evaluation/env_infos/X_pos_2 Mean               359.035
evaluation/env_infos/X_pos_2 Std                132.578
evaluation/env_infos/X_pos_2 Max                580
evaluation/env_infos/X_pos_2 Min                121
time/data storing (s)                             0.000143691
time/evaluation sampling (s)                      4.7883
time/exploration sampling (s)                     0.00851269
time/logging (s)                                  0.0557408
time/saving (s)                                   0.00679168
time/training (s)                                 4.43137
time/epoch (s)                                    9.29086
time/total (s)                                   20.0185
Epoch                                             1
---------------------------------------------  --------------
2019-06-08 21:14:22.679114 Pacific Daylight Time | [h:\work\psu\ece 510(deep learning theory

evaluation/env_infos/final/X_pos_2 Std           129.747
evaluation/env_infos/final/X_pos_2 Max           580
evaluation/env_infos/final/X_pos_2 Min           120
evaluation/env_infos/initial/X_pos_2 Mean        341.488
evaluation/env_infos/initial/X_pos_2 Std         129.747
evaluation/env_infos/initial/X_pos_2 Max         580
evaluation/env_infos/initial/X_pos_2 Min         120
evaluation/env_infos/X_pos_2 Mean                347.585
evaluation/env_infos/X_pos_2 Std                 128.524
evaluation/env_infos/X_pos_2 Max                 580
evaluation/env_infos/X_pos_2 Min                 120
time/data storing (s)                              0.000136712
time/evaluation sampling (s)                       4.6502
time/exploration sampling (s)                      0.00766656
time/logging (s)                                   0.0499353
time/saving (s)                                    0.00794819
time/training (s)                                  5.01254
time/epoch (s)                  

evaluation/env_infos/initial/Velocity_y Min        1
evaluation/env_infos/Velocity_y Mean               5.7148
evaluation/env_infos/Velocity_y Std                2.85317
evaluation/env_infos/Velocity_y Max               10
evaluation/env_infos/Velocity_y Min                1
evaluation/env_infos/final/X_pos_2 Mean          355.646
evaluation/env_infos/final/X_pos_2 Std           135.305
evaluation/env_infos/final/X_pos_2 Max           580
evaluation/env_infos/final/X_pos_2 Min           121
evaluation/env_infos/initial/X_pos_2 Mean        355.646
evaluation/env_infos/initial/X_pos_2 Std         135.305
evaluation/env_infos/initial/X_pos_2 Max         580
evaluation/env_infos/initial/X_pos_2 Min         121
evaluation/env_infos/X_pos_2 Mean                360.475
evaluation/env_infos/X_pos_2 Std                 135.053
evaluation/env_infos/X_pos_2 Max                 580
evaluation/env_infos/X_pos_2 Min                 121
time/data storing (s)                              0.000139997
t

...

evaluation/path length Max                         10
evaluation/path length Min                          7
evaluation/Rewards Mean                             0.240696
evaluation/Rewards Std                              2.78851
evaluation/Rewards Max                              1
evaluation/Rewards Min                            -10
evaluation/Returns Mean                             2.21547
evaluation/Returns Std                              5.93764
evaluation/Returns Max                             10
evaluation/Returns Min                             -4
evaluation/Actions Mean                            61.5366
evaluation/Actions Std                             21.7646
evaluation/Actions Max                             88
evaluation/Actions Min                              9
evaluation/Num Paths                              543
evaluation/Average Returns                          2.21547
evaluation/env_infos/final/actions Mean             0
evaluation/env_infos/final/actions Std   

exploration/env_infos/initial/X_pos_2 Min         123
exploration/env_infos/X_pos_2 Mean                514.5
exploration/env_infos/X_pos_2 Std                 130.5
exploration/env_infos/X_pos_2 Max                 558
exploration/env_infos/X_pos_2 Min                 123
evaluation/num steps total                     164878
evaluation/num paths total                      17785
evaluation/path length Mean                         9.61538
evaluation/path length Std                          0.809887
evaluation/path length Max                         10
evaluation/path length Min                          7
evaluation/Rewards Mean                             0.725
evaluation/Rewards Std                              1.71737
evaluation/Rewards Max                              1
evaluation/Rewards Min                            -10
evaluation/Returns Mean                             6.97115
evaluation/Returns Std                              5.40167
evaluation/Returns Max                     

exploration/env_infos/Velocity_y Max                6
exploration/env_infos/Velocity_y Min                2
exploration/env_infos/final/X_pos_2 Mean          477.5
exploration/env_infos/final/X_pos_2 Std            54.5
exploration/env_infos/final/X_pos_2 Max           532
exploration/env_infos/final/X_pos_2 Min           423
exploration/env_infos/initial/X_pos_2 Mean        477.5
exploration/env_infos/initial/X_pos_2 Std          54.5
exploration/env_infos/initial/X_pos_2 Max         532
exploration/env_infos/initial/X_pos_2 Min         423
exploration/env_infos/X_pos_2 Mean                444.8
exploration/env_infos/X_pos_2 Std                  43.6
exploration/env_infos/X_pos_2 Max                 532
exploration/env_infos/X_pos_2 Min                 423
evaluation/num steps total                     169874
evaluation/num paths total                      18405
evaluation/path length Mean                         8.05806
evaluation/path length Std                          0.996696
eva

exploration/env_infos/final/Velocity_y Std          0
exploration/env_infos/final/Velocity_y Max          1
exploration/env_infos/final/Velocity_y Min          1
exploration/env_infos/initial/Velocity_y Mean       1
exploration/env_infos/initial/Velocity_y Std        0
exploration/env_infos/initial/Velocity_y Max        1
exploration/env_infos/initial/Velocity_y Min        1
exploration/env_infos/Velocity_y Mean               1
exploration/env_infos/Velocity_y Std                0
exploration/env_infos/Velocity_y Max                1
exploration/env_infos/Velocity_y Min                1
exploration/env_infos/final/X_pos_2 Mean          265
exploration/env_infos/final/X_pos_2 Std             0
exploration/env_infos/final/X_pos_2 Max           265
exploration/env_infos/final/X_pos_2 Min           265
exploration/env_infos/initial/X_pos_2 Mean        265
exploration/env_infos/initial/X_pos_2 Std           0
exploration/env_infos/initial/X_pos_2 Max         265
exploration/env_infos/initia

KeyboardInterrupt: 

In [4]:
env = gym.make('Physics_RL-v0')
env.reset()

(10, 174, 421, 10)

In [21]:
#env.observation_space.low.size
env.step(10)

((180, 174, 421, 95),
 0,
 False,
 {'Velocity_y': 5,
  'X_pos_2': 421,
  'Y_Pos_1': 278,
  'actions': [10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10]})